In [ ]:
!pip install pypulseq==1.3.1.post1 &> /dev/null
!pip install MRzeroCore &> /dev/null
!pip install torchkbnufft &> /dev/null
!wget https://github.com/MRsources/MRzero-Core/raw/main/documentation/playground_mr0/numerical_brain_cropped.mat &> /dev/null

(mr0_CS_radial_seq)=
# Compressed Sensing - radial

In [ ]:
# %% S0. SETUP env
from skimage.restoration import denoise_tv_chambolle
import pywt
import MRzeroCore as mr0
import pypulseq as pp
import numpy as np
import torch
from matplotlib import pyplot as plt
plt.rcParams['figure.figsize'] = [10, 5]
plt.rcParams['figure.dpi'] = 100 # 200 e.g. is really fine, but slower

experiment_id = 'exF02_undersampled_radial'


# %% S1. SETUP sys

# choose the scanner limits
system = pp.Opts(
    max_grad=28, grad_unit='mT/m', max_slew=150, slew_unit='T/m/s',
    rf_ringdown_time=20e-6, rf_dead_time=100e-6,
    adc_dead_time=20e-6, grad_raster_time=50 * 10e-6
)


# %% S2. DEFINE the sequence
seq = pp.Sequence()

# Define FOV and resolution
fov = 220e-3
slice_thickness = 8e-3
sz = [128, 128]    # spin system size / resolution
Nread = sz[0]    # frequency encoding steps/samples
Nphase = 61      # phase encoding steps/samples, i.e. number of radial spokes

# Define rf events
rf1, _, _ = pp.make_sinc_pulse(
    flip_angle=6 * np.pi / 180, duration=1e-3,
    slice_thickness=slice_thickness, apodization=0.5, time_bw_product=4,
    system=system, return_gz=True
)
# rf1 = pp.make_block_pulse(flip_angle=90 * np.pi / 180, duration=1e-3, system=system)

# Define other gradients and ADC events
gx = pp.make_trapezoid(channel='x', flat_area=Nread / fov, flat_time=5e-3, system=system)
gy = pp.make_trapezoid(channel='y', flat_area=Nread / fov, flat_time=5e-3, system=system)

gx_pre = pp.make_trapezoid(channel='x', area=-gx.area / 2, duration=1e-3, system=system)
gy_pre = pp.make_trapezoid(channel='y', area=-gy.area / 2, duration=1e-3, system=system)

os = 1
adc = pp.make_adc(num_samples=Nread * os, duration=5e-3, phase_offset=0 * np.pi / 180, delay=gx.rise_time, system=system)

rf_phase = 180
rf_inc = 180

# ======
# CONSTRUCT SEQUENCE
# ======
sdel = 1e-0

rf0, _, _ = pp.make_sinc_pulse(
    flip_angle=6 / 2 * np.pi / 180, duration=1e-3,
    slice_thickness=slice_thickness, apodization=0.5, time_bw_product=4,
    system=system, return_gz=True
)
seq.add_block(rf0)
seq.add_block(pp.make_delay(3e-3))

for ii in range(-Nphase // 2, Nphase // 2):  # e.g. -64:63
    rf1.phase_offset = rf_phase / 180 * np.pi   # set current rf phase

    adc.phase_offset = rf_phase / 180 * np.pi  # follow with ADC
    # increment additional pahse
    rf_phase = divmod(rf_phase + rf_inc, 360.0)[1]

    seq.add_block(rf1)

    gx = pp.make_trapezoid(channel='x', flat_area=-Nread * np.sin(ii / Nphase * np.pi) / fov + 1e-9, flat_time=5e-3, system=system)
    gy = pp.make_trapezoid(channel='y', flat_area=Nread * np.cos(ii / Nphase * np.pi) / fov + 1e-9, flat_time=5e-3, system=system)

    gx_pre = pp.make_trapezoid(channel='x', area=-gx.area / 2, duration=1e-3, system=system)
    gy_pre = pp.make_trapezoid(channel='y', area=-gy.area / 2, duration=1e-3, system=system)

    seq.add_block(gx_pre, gy_pre)
    seq.add_block(adc, gx, gy)
    # seq.add_block(adc,gx,gy)
    seq.add_block(gx_pre, gy_pre)
    # seq.add_block(make_delay(10))


# %% S3. CHECK, PLOT and WRITE the sequence  as .seq
# Check whether the timing of the sequence is correct
ok, error_report = seq.check_timing()
if ok:
    print('Timing check passed successfully')
else:
    print('Timing check failed. Error listing follows:')
    [print(e) for e in error_report]

# PLOT sequence
sp_adc, t_adc = mr0.util.pulseq_plot(seq, clear=False)

# Prepare the sequence output for the scanner
seq.set_definition('FOV', [fov, fov, slice_thickness])
seq.set_definition('Name', 'gre')
seq.write(experiment_id + '.seq')


# %% S4: SETUP SPIN SYSTEM/object on which we can run the MR sequence external.seq from above

if 1:
    # (i) load a phantom object from file
    obj_p = mr0.VoxelGridPhantom.load_mat('numerical_brain_cropped.mat')
    obj_p = obj_p.interpolate(sz[0], sz[1], 1)
    # Manipulate loaded data
    obj_p.T2dash[:] = 30e-3
    obj_p.D *= 0
    obj_p.B0 *= 0
else:
    # or (ii) set phantom  manually to a pixel phantom. Coordinate system is [-0.5, 0.5]^3
    obj_p = mr0.CustomVoxelPhantom(
        pos=[[-0.4, -0.4, 0], [-0.4, -0.2, 0], [-0.3, -0.2, 0], [-0.2, -0.2, 0], [-0.1, -0.2, 0]],
        PD=[1.0, 1.0, 0.5, 0.5, 0.5],
        T1=1.0,
        T2=0.1,
        T2dash=0.1,
        D=0.0,
        voxel_size=0.1,
        voxel_shape="box"
    )

obj_p.plot()
# Convert Phantom into simulation data
obj_p = obj_p.build()


# %% S5:. SIMULATE  the external.seq file and add acquired signal to ADC plot

# Read in the sequence
seq0 = mr0.Sequence.import_file(experiment_id + '.seq')
seq0.plot_kspace_trajectory()
kspace_loc = seq0.get_kspace()
# Simulate the sequence
graph = mr0.compute_graph(seq0, obj_p, 200, 1e-3)
signal = mr0.execute_graph(graph, seq0, obj_p, print_progress=False)

# PLOT sequence with signal in the ADC subplot
sp_adc, t_adc = mr0.util.pulseq_plot(seq, clear=True, signal=signal.numpy())

kspace_adc = torch.reshape((signal), (Nphase, Nread)).clone().t()

In [ ]:
print('kspace_loc shape: ', kspace_loc.shape)
print('kspace_adc shape: ', kspace_adc.shape)

f, ax = plt.subplots(1, 1, figsize=(6, 6))
ax.imshow(np.log10(abs(kspace_adc)), cmap='gray')
ax.set_xlabel('radial spoke index')
ax.set_ylabel('readout')

In [ ]:
traj = torch.reshape(kspace_loc, (Nphase, Nread, kspace_loc.shape[-1]))
traj = traj[..., :2]

traj_scale = traj/Nread * 2
traj_convert = torch.reshape(traj_scale, (-1, 2)).transpose(1, 0)

print('traj_scale shape: ', traj_scale.shape)
print('traj_convert shape: ', traj_convert.shape)

kspace_res = torch.view_as_real(kspace_adc.transpose(1, 0))
kspace_res = torch.reshape(kspace_res, (1, 1, -1, 2))

print('kspace_res shape: ', kspace_res.shape)

In [ ]:
import scipy.interpolate
grid = kspace_loc[:, :2]
Nx = sz[0]
Ny = sz[1]

X, Y = np.meshgrid(np.linspace(0, Nx - 1, Nx) - Nx / 2,
                    np.linspace(0, Ny - 1, Ny) - Ny / 2)
grid = np.double(grid.numpy())
grid[np.abs(grid) < 1e-3] = 0

# plot every 8 radial spokes
f, ax = plt.subplots(1, 1, figsize=(9, 9))
ax.plot(traj[::8, :Nx, 0].ravel(), traj[::8, :Nx, 1].ravel(), 'rx', markersize=3)
ax.plot(X, Y, 'k.', markersize=2)
plt.show()

print(np.amin(grid[:, 0]), np.amax(grid[:, 0]))
print(np.amin(grid[:, 1]), np.amax(grid[:, 1]))

NUFFT recon with density compensation

  * Zhengguo Tan <zhengguo.tan@gmail.com>



In [ ]:
import torchkbnufft as tkbn

# compute density compensation function
dcf = (traj_convert[0, ...]**2 + traj_convert[1, ...]**2)**0.5
dcf = dcf.reshape(1, -1).repeat(2, 1).transpose(1, 0)

print('dcf shape: ', dcf.shape)


img_shape = [Nread] * 2


# define nufft adjoint operator
nufft_adj = tkbn.KbNufftAdjoint(im_size=img_shape)

img_dcf_tensor = nufft_adj(kspace_res * dcf, traj_convert)
img_tensor = nufft_adj(kspace_res, traj_convert)

print('nufft_adj -> image shape: ', img_tensor.shape)


# define nufft forward operator
nufft_fwd = tkbn.KbNufft(im_size=img_shape)

kspace_fwd = nufft_fwd(img_tensor, traj_convert)

kspace_fwd_res = torch.reshape(kspace_fwd, (1, 1, Nphase, Nread, 2))

ksp_cplx = torch.view_as_complex(kspace_fwd_res).transpose(-1, -2).cpu().detach().numpy()

print('nufft_fwd -> kspace shape: ', kspace_fwd.shape)

img = img_dcf_tensor.cpu().detach().numpy()

img_cplx = img[..., 0] + 1j * img[..., 1]
img_cplx = np.flip(np.swapaxes(img_cplx, -1, -2), -2)

f, ax = plt.subplots(1, 2, figsize=(12, 6))
ax[0].imshow(abs(np.squeeze(img_cplx)), cmap='gray')
ax[1].imshow(np.log10(abs(np.squeeze(ksp_cplx))), cmap='gray')
plt.show()

Compressed sensing recon for radial sampling

  * Zhengguo Tan <zhengguo.tan@gmail.com>
  

In [ ]:
def soft_thresh(input, lamda):

    abs_input = abs(input)

    sign = np.true_divide(input, abs_input,
                          out=np.zeros_like(input), where=abs_input!=0)

    magn = abs_input - lamda
    magn = (abs(magn) + magn) / 2

    return magn * sign


def prox_wav(input, lamda):

    if torch.is_tensor(input):
        input = input.cpu().detach().numpy()

    # pywt outputs numpy arrays
    cA, (cH, cV, cD) = pywt.dwt2(input, 'db4')

    cA_t = soft_thresh(cA, lamda)
    cH_t = soft_thresh(cH, lamda)
    cV_t = soft_thresh(cV, lamda)
    cD_t = soft_thresh(cD, lamda)

    wav_coef = cA_t, (cH_t, cV_t, cD_t)

    output = pywt.idwt2(wav_coef, 'db4')

    return torch.tensor(output)


# compute maximal eigenvalue:
x = torch.randn(size=img_tensor.shape, dtype=img_tensor.dtype)
for n in range(30):
    y = nufft_adj(nufft_fwd(x, traj_convert), traj_convert)
    max_eig = torch.linalg.norm(y).ravel()
    x = y / max_eig

    print(max_eig)


# Gradient method
Niter = 200

alpha = (1 / max_eig).cpu().detach().numpy().item()
lamda = 0.001

x = torch.zeros_like(img_tensor)

for n in range(Niter):

    x_old = x.clone()

    r = nufft_fwd(x, traj_convert) - kspace_res
    g = nufft_adj(r, traj_convert)

    x = prox_wav(x - alpha * g, alpha * lamda)


    resid = (torch.linalg.norm(x - x_old).ravel()).ravel()

    print('> iter ' + str(n).zfill(4) + ' residuum ' + str(resid[0]))

Plot results

In [ ]:
recon = torch.view_as_complex(x).cpu().detach().numpy()
recon = np.flip(np.swapaxes(recon, -1, -2), -2)

R1 = abs(np.squeeze(img_cplx))
R2 = abs(np.squeeze(recon))

f, ax = plt.subplots(1, 2, figsize=(12, 6))
ax[0].imshow(R1, cmap='gray', vmin=0)  # [32:96, 32:96]
ax[0].set_title('NUFFT')

ax[1].imshow(R2, cmap='gray', vmin=0)
ax[1].set_title('Compressed Sensing')
plt.show()